### Configurar Qiskit para usar la Computadora Cuántica

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from dotenv import load_dotenv
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [ ]:
# Cargar variables de entorno
load_dotenv(override=True)

# Cargar tu API Key desde el entorno .env
backend_name = os.getenv("IBMQ_BACKEND")
api_token = os.getenv("IBMQ_API_TOKEN")

if not api_token:
    raise ValueError("API Token no encontrado. Asegúrate de configurarlo en el archivo .env.")

if not backend_name:
    raise ValueError("Backend no encontrado. Asegúrate de configurarlo en el archivo .env.")


QiskitRuntimeService.save_account(channel="ibm_quantum", token=api_token, overwrite=True)

# Conectar al servicio
service = QiskitRuntimeService()


# Seleccionar el backend
backend = service.backend(backend_name)
print(f"Usando backend: {backend}")



### Implementar la QFT en Qiskit en Computadora Cuántica Real


In [ ]:

# Definir el circuito QFT
def qft_circuit(n):
    """Crear un circuito QFT para n qubits."""
    qc = QuantumCircuit(n)
    for qubit in range(n):
        qc.h(qubit)  # Compuerta Hadamard
        for k in range(qubit + 1, n):
            angle = np.pi / (2 ** (k - qubit))
            qc.cp(angle, qubit, k)  # Rotaciones controladas
    qc.barrier()
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)  # Invertir qubits
    return qc

# Crear el circuito QFT para 8 qubits
n_qubits = 8
qc = qft_circuit(n_qubits)
qc.measure_all()

# Transpilar el circuito para el backend seleccionado
print("Transpilando el circuito para el backend seleccionado...")
transpiled_circuit = transpile(qc, backend)

# Ejecutar el circuito en el backend cuántico
print("Ejecutando el circuito en el backend cuántico...")
job = backend.run(transpiled_circuit, shots=4096)

# Esperar los resultados
print("Esperando resultados del backend cuántico...")
result = job.result()
counts = result.get_counts()
print("Resultados obtenidos del backend cuántico:", counts)

# Normalizar los resultados para que sean proporciones
total_shots = sum(counts.values())  # Total de mediciones realizadas
normalized_counts = {k: v / total_shots for k, v in counts.items()}
print("Normalized counts:", normalized_counts)

# Filtrar los estados con baja frecuencia para mejorar la claridad
threshold = 0.001  # Reducir el umbral mínimo de frecuencia (0.1%)
filtered_counts = {k: v for k, v in normalized_counts.items() if v >= threshold}

# Si no hay datos después del filtrado, mostrar todos los datos
if not filtered_counts:
    print("No hay estados con frecuencia >= 0.1%. Mostrando todos los datos.")
    filtered_counts = normalized_counts

# Mostrar advertencia si muchos datos fueron eliminados
if len(filtered_counts) < len(normalized_counts):
    print(f"Se filtraron {len(normalized_counts) - len(filtered_counts)} estados con frecuencia < {threshold * 100}%.")

# Ordenar y graficar los resultados filtrados
sorted_counts = dict(sorted(filtered_counts.items(), key=lambda item: item[0]))  # Ordenar por estado binario
plt.figure(figsize=(12, 6))  # Ajustar el tamaño de la figura
plot_histogram(sorted_counts, bar_labels=True)  # Mostrar etiquetas en las barras

# Personalizar la visualización
plt.title("Resultados de la Transformada de Fourier Cuántica (QFT)", fontsize=16)
plt.xlabel("Estados medidos (binario)", fontsize=14)
plt.ylabel("Frecuencia de conteo (normalizada)", fontsize=14)
plt.xticks(rotation=90, fontsize=10)  # Rotar etiquetas del eje x para claridad
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Agregar cuadrícula en el eje y
plt.tight_layout()  # Ajustar el espaciado automáticamente para evitar superposiciones
plt.show()